In [6]:
import mechanize
import http.cookiejar as cookielib
from bs4 import BeautifulSoup
from datetime import datetime
import time
import pandas

In [7]:
cj = cookielib.CookieJar()
br = mechanize.Browser()
br.set_cookiejar(cj)

In [8]:
br.open("https://op.responsive.net/lt/jackson/entry.html") #Replace this url with yours
br.select_form(nr=0)
br.form['id'] = 'kastam' # Replace this id with yours 
br.form['password'] = 'fano123' # Replace this password with yours
br.submit()

<response_seek_wrapper at 0x2d3e0bb1c60 whose wrapped object = <closeable_response at 0x2d3e0bb1a50 whose fp = <_io.BufferedReader>>>

In [9]:
url_list = ["CASH","JOBIN","JOBQ","S1Q","S2Q","S3Q","S1UTIL","S2UTIL","S3UTIL"]
url_list_4col = ["JOBT","JOBREV","JOBOUT"]
LF_DATA = {}

In [11]:
while True:
    lf_url = "https://op.responsive.net/Littlefield/Plot?data=INV&x=all"
    soup = BeautifulSoup(br.open(lf_url), "lxml")

    data = soup.findAll("script")[6].string
    data = data.split("\n")[4].split("'")[3].split()

    counter = 1
    LF_DATA = {}

    for i in data:
     if counter % 2 == 1:
      counter += 1
      day = float(i)
      LF_DATA[day] = []
     elif counter % 2 == 0:
      row_data = [float(i)]
      LF_DATA[day].extend(row_data)
      counter += 1

    for url in url_list:
     lf_url = "http://op.responsive.net/Littlefield/Plot?data=%s&x=all" % url
     soup = BeautifulSoup(br.open(lf_url), "lxml")
     data = soup.findAll("script")[6].string
     data = data.split("\n")[4].split("'")[3].split()
     counter = 1
     for i in data:
      if counter % 2 == 0:
       day = counter/2
       LF_DATA[day].append(float(i))
       counter += 1
      else:
       counter +=1

    for url in url_list_4col:
     lf_url = "http://op.responsive.net/Littlefield/Plot?data=%s&x=all" % url
     soup = BeautifulSoup(br.open(lf_url), "lxml")
     data = soup.findAll("script")[6].string
     data0 = data.split("\n")[4].split("'")[5].split()
     data1 = data.split("\n")[5].split("'")[5].split()
     data2 = data.split("\n")[6].split("'")[5].split()

     counter = 1

     for i in data0:
      if counter % 2 == 0:
       day = counter/2
       LF_DATA[day].append(float(i))
       counter += 1
      else:
       counter +=1

     counter = 1

     for i in data1:
      if counter % 2 == 0:
       day = counter/2
       LF_DATA[day].append(float(i))
       counter += 1
      else:
       counter +=1

     counter = 1

     for i in data2:
      if counter % 2 == 0:
       day = counter/2
       LF_DATA[day].append(float(i))
       counter += 1
      else:
       counter +=1

    dummy_data = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for key, value in LF_DATA.items():
     if len(value) < 19:
      value.extend(dummy_data)

    headers = ["INV","CASH","JOBIN","JOBQ","S1Q","S2Q","S3Q","S1UTIL","S2UTIL","S3UTIL","JOBTC1","JOBTC2","JOBTC3","JOBREVC1","JOBREVC2","JOBREVC3","JOBOUTC1","JOBOUTC2","JOBOUTC3"]

    df = pandas.DataFrame.from_dict(LF_DATA, orient="index")
    df.columns = headers
    df.sort_index(inplace=True)
    df["Backlog"] = df["JOBIN"].cumsum() - df["JOBOUTC1"].cumsum() - df["JOBOUTC2"].cumsum() - df["JOBOUTC3"].cumsum()
    df.index.rename('DAY', inplace=True)

    df.index = df.index.astype(int)
    df= df[~df.index.duplicated(keep='first')]
    
    df.to_excel("Data Extract 10_06_2023_12_35_04.xlsx")
    
    time.sleep(42 * 60)